In [1]:
import pandas as pd

# 2024 Fantasy Football Analysis!

This year we are starting off with some lessons learned from 2023.

I will be focusing specifically this year on offensive positions.

Things we want to implement this year:
1. A UI to make the picking process more seamless
2. Dynamic picking process that matches the positions available each week.

Things not considering:
1. laterals

### EDA!

Columns we Will Entertain:

1. ID Columns
    - game_id
    - play_id
    - home_team
    - away_team
    - season_type
    - posteam
    - defteam
    - play_type
    - passer_player_id
    - passer_player_name
    - receiver_player_id
    - receiver_player_name
    - rusher_player_id
    - rusher_player_name
    - season
    - div_game
    - home_coach
    - away_coach
    

2. Analysis Columns
    - yards_gained
    - incomplete_pass
    - interception
    - rush_attempt
    - pass_attempt
    - sack
    - touchdown
    - pass_touchdown
    - rush_touchdown
    - passing_yards
    - receiving_yards
    - rushing_yards
    - series_result
    - sack_player_id
    - sack_player_name
    - fumbled_1_player_id
    - fumbled_1_player_name
    

In [16]:
pbp_data = (pd.read_csv('pbp_data.csv')
            .drop(columns=['Unnamed: 0']))[['game_id',
                                            'play_id',
                                            'home_team',
                                            'away_team',
                                            'season_type',
                                            'posteam',
                                            'defteam',
                                            'play_type',
                                            'passer_player_id',
                                            'passer_player_name',
                                            'receiver_player_id',
                                            'receiver_player_name',
                                            'rusher_player_id',
                                            'rusher_player_name',
                                            'season',
                                            'div_game',
                                            'home_coach',
                                            'away_coach',
                                            'yards_gained',
                                            'incomplete_pass',
                                            'interception',
                                            'rush_attempt',
                                            'pass_attempt',
                                            'sack',
                                            'touchdown',
                                            'pass_touchdown',
                                            'rush_touchdown',
                                            'passing_yards',
                                            'receiving_yards',
                                            'rushing_yards',
                                            'series_result',
                                            'sack_player_id',
                                            'sack_player_name',
                                            'fumbled_1_player_id',
                                            'fumbled_1_player_name']]

/var/folders/cc/0nj_vbl54hlbtpxkwz1vc5yc0000gn/T/ipykernel_2739/1468954226.py:1: DtypeWarning: Columns (46,180,181,183,184,190,191,194,195,196,197,198,199,204,205,206,207,210,211,214,215,219,220,221,223,225,227,234,235,236,237,238,239,244,245,246,249,250,254,255,256,261,263,264,267,268,269,270,278,284,285,303,333) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data = (pd.read_csv('pbp_data.csv')


In [17]:
pbp_data.head(5)

,game_id,play_id,home_team,away_team,season_type,posteam,defteam,play_type,passer_player_id,passer_player_name,...,pass_touchdown,rush_touchdown,passing_yards,receiving_yards,rushing_yards,series_result,sack_player_id,sack_player_name,fumbled_1_player_id,fumbled_1_player_name
0,2018_01_ATL_PHI,1,PHI,ATL,REG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,First down,NaN,NaN,NaN,NaN
1,2018_01_ATL_PHI,37,PHI,ATL,REG,ATL,PHI,kickoff,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,First down,NaN,NaN,NaN,NaN
2,2018_01_ATL_PHI,52,PHI,ATL,REG,ATL,PHI,no_play,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,First down,NaN,NaN,NaN,NaN
3,2018_01_ATL_PHI,75,PHI,ATL,REG,ATL,PHI,pass,00-0026143,M.Ryan,...,0.0,0.0,10.0,10.0,NaN,First down,NaN,NaN,NaN,NaN
4,2018_01_ATL_PHI,104,PHI,ATL,REG,ATL,PHI,run,NaN,NaN,...,0.0,0.0,NaN,NaN,11.0,First down,NaN,NaN,NaN,NaN


In [18]:
pbp_data.tail(5)

,game_id,play_id,home_team,away_team,season_type,posteam,defteam,play_type,passer_player_id,passer_player_name,...,pass_touchdown,rush_touchdown,passing_yards,receiving_yards,rushing_yards,series_result,sack_player_id,sack_player_name,fumbled_1_player_id,fumbled_1_player_name
291088,2023_22_SF_KC,4791,KC,SF,POST,KC,SF,run,NaN,NaN,...,0.0,0.0,NaN,NaN,19.0,First down,NaN,NaN,NaN,NaN
291089,2023_22_SF_KC,4813,KC,SF,POST,KC,SF,run,NaN,NaN,...,0.0,0.0,NaN,NaN,3.0,First down,NaN,NaN,NaN,NaN
291090,2023_22_SF_KC,4835,KC,SF,POST,KC,SF,pass,00-0033873,P.Mahomes,...,0.0,0.0,7.0,7.0,NaN,First down,NaN,NaN,NaN,NaN
291091,2023_22_SF_KC,4860,KC,SF,POST,KC,SF,pass,00-0033873,P.Mahomes,...,1.0,0.0,3.0,3.0,NaN,Touchdown,NaN,NaN,NaN,NaN
291092,2023_22_SF_KC,4881,KC,SF,POST,KC,SF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Touchdown,NaN,NaN,NaN,NaN


In [19]:
list(pbp_data.columns)

['game_id',
 'play_id',
 'home_team',
 'away_team',
 'season_type',
 'posteam',
 'defteam',
 'play_type',
 'passer_player_id',
 'passer_player_name',
 'receiver_player_id',
 'receiver_player_name',
 'rusher_player_id',
 'rusher_player_name',
 'season',
 'div_game',
 'home_coach',
 'away_coach',
 'yards_gained',
 'incomplete_pass',
 'interception',
 'rush_attempt',
 'pass_attempt',
 'sack',
 'touchdown',
 'pass_touchdown',
 'rush_touchdown',
 'passing_yards',
 'receiving_yards',
 'rushing_yards',
 'series_result',
 'sack_player_id',
 'sack_player_name',
 'fumbled_1_player_id',
 'fumbled_1_player_name']